# Otsu thresholding.
Joshua Stough
DIP

Short example demonstrating Otsu's method for image foreground
background thresholding. Simply put, the goal is to choose the
threshold that maximizes the between class variance.
Ref: Gonzalez/Woods, DIP. Section 10.3
[Wiki](https://en.wikipedia.org/wiki/Otsu%27s_method)

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#I = plt.imread('rice.png') #Popular example image
I = plt.imread('germanOtsuWiki.jpg') #Wiki example
#I = plt.imread('underExposed.jpg')


#If it's a multi-channel image, try to get it in [0,255]
if (len(I.shape) > 2):
    Ig = 0.2989 * I[..., 0] + 0.5870 * I[..., 1] + 0.1140 * I[..., 2]
    I = Ig

    I = I/I.ravel().max()
    I = 255*I
    I = np.round(I)
    I[I>255] = 255
    I = I.copy().astype('uint8')

In [4]:
plt.figure()
plt.imshow(I, cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:

#Using the nomenclature of the book:
hist, bins = np.histogram(I.ravel(), np.arange(257))

p = hist/sum(hist)  #probability density, 10-48
P_1 = np.cumsum(p)    #cumulative sum 10-49

#intermediate, intensity times prob, i*p_i
ip = np.multiply(bins[:-1], p)

#m(k), the cumulative mean up to k, for all k. 10-53
mk = np.cumsum(ip)

#m_G, global mean intensity, 10-54
#should be equivalent to I.ravel().mean()
m_G = ip.sum()

#We're going to compute the between class variance
#as a function of every possible threshold value.
numerator = np.square(m_G*P_1 - mk)
denom = np.multiply(P_1, 1.0 - P_1)

#This would possibly allow division by zero.
#sig_B = np.divide(numerator, denom)
#so

sig_B = np.zeros(len(mk))
eps = np.finfo(float).eps #pretty much zero
#10-62
sig_B[denom > eps] = np.divide(numerator[denom>eps], denom[denom>eps])


#Get the max
opt = np.argmax(sig_B)

f, ax = plt.subplots(1,4, figsize=(15,3))

ax[0].imshow(I, cmap = 'gray')
ax[0].set_title('Original Image')

ax[1].bar(bins[:-1], hist)
ax[1].set_title('Image Histogram')

ax[2].plot(bins[:-1], sig_B)
ax[2].set_title('Inter-Class Variance')

F = I>opt #Which pixels are above the threshold.
ax[3].imshow(F.astype(float))
ax[3].set_title('Threshold=%d' % opt)


#Compute the separability, eta. 10-61
sig_G = np.multiply(np.square(bins[:-1]-m_G), p).sum()  #10-58
etaOpt = sig_B[opt]/sig_G   #10-61


#Image with just foreground.
If = I.copy()
If[~F] = 0

f, ax2 = plt.subplots(1,2, figsize=(10,5), sharex=True, sharey=True)
ax2[0].imshow(I, cmap = 'gray')
ax2[0].set_title('Original Image')

ax2[1].imshow(If, cmap = 'gray')
ax2[1].set_title('Foreground Image\nSeparability %5.3f' % etaOpt)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Foreground Image\nSeparability 0.782')